In [250]:
import os 
import pybedtools as pbt
import pandas as pd
import numpy as np
import subprocess as sp
os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge/')
pbt.set_bedtools_path('/mnt/BioApps/bedtools/bin/')
bgzip = '/mnt/BioApps/tabix/tabix-0.2.6/bgzip'
tabix = '/mnt/BioApps/tabix/tabix-0.2.6/tabix'

outdir = 'results/main/loop_analysis/washU/'
os.makedirs(outdir, exist_ok=True)

bedpe_6cols = ['chrA', 'startA', 'endA', 'chrB', 'startB', 'endB']
bedpe_10cols = ['chrA', 'startA', 'endA', 'chrB', 'startB', 'endB', 'name', 'score', 'strand1', 'strand2']

## Intersect loops with SNPs and Genes

In [251]:
# load the colocalization data
coloc_fn = 'results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/'
coloc_fn += 'BLUEPRINT_eQTL_Monocyte/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed'
coloc = pd.read_table(coloc_fn)

In [252]:
# extract the most significant according the H4 
coloc_sig = coloc[coloc['pp_H4_Coloc_Summary'] > 0.75]
coloc_sig = coloc_sig[['chr', 'pos', 'rs_id', 'variant_id']]
coloc_sig.rename(columns={'pos': 'end'}, inplace=True)
coloc_sig.loc[:, 'start'] = coloc_sig.loc[:, 'end'] - 1
coloc_sig = coloc_sig[['chr', 'start', 'end', 'rs_id', 'variant_id']]
coloc_sig = pbt.BedTool.from_dataframe(coloc_sig)

In [254]:
print('In total there are {} colocalized SNPs.'.format(len(coloc_sig)))

In total there are 18 colocalized SNPs.


In [99]:
# load the loop data
loop_fn = 'results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/CD4N/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed'
loops = pd.read_table(loop_fn)
tmp_loops = loops[['chr1', 's1', 'e1', 'chr2', 's2', 'e2']]
tmp_loops.rename(columns={'p': 'score'}, inplace=True)
tmp_loops.loc[:, 'name'] = '.'
tmp_loops.loc[:, 'score'] = loops['p']
tmp_loops.loc[:, 'strand1'] = '.'
tmp_loops.loc[:, 'strand2'] = '.'
loops = pbt.BedTool.from_dataframe(tmp_loops)

/mnt/BioHome/jreyna/.local/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/mnt/BioHome/jreyna/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/mnt/BioHome/jreyna/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [258]:
print('FitHiChIP found {} significant loops.'.format(loops.count()))

FitHiChIP found 114421 significant loops.


In [259]:
# extract loops where an anchor overlaps with a SNP
snp_loops = loops.pairtobed(coloc_sig)

In [264]:
snp_loops_tmp = snp_loops.to_dataframe(disable_auto_names=True, header=None)
nsnp_loops = sum(snp_loops_tmp.duplicated(subset=[0,1,2,3,4,5]))
print('There are {} loops with colocalized SNPs.'.format(nsnp_loops))

There are 12 loops with colocalized SNPs


In [265]:
# load the gene coords
genes_fn = 'results/refs/ensembl/Homo_sapiens.GRCh38.104.bed'
genes = pbt.BedTool(genes_fn)

In [266]:
# extract loops where an anchor overlaps with a SNP
gs_loops = snp_loops.pairtobed(genes)
gs_loops = gs_loops.to_dataframe(disable_auto_names=True, header=None)

In [267]:
gs_loops

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,chr1,192535000,192540000,chr1,193025000,193030000,.,3.378864e-07,.,.,...,192538495,192538496,rs2760530,rs2760530,chr1,192517190,192538102,ENSG00000236069,1,ENSG00000236069
1,chr1,192535000,192540000,chr1,193025000,193030000,.,3.378864e-07,.,.,...,192538495,192538496,rs2760530,rs2760530,chr1,192167786,192957713,ENSG00000285280,1,ENSG00000285280
2,chr1,192535000,192540000,chr1,193025000,193030000,.,3.378864e-07,.,.,...,192538495,192538496,rs2760530,rs2760530,chr1,193012250,193060080,UCHL5,1,ENSG00000116750
3,chr11,62605000,62610000,chr11,64105000,64110000,.,1.006396e-07,.,.,...,64107476,64107477,rs479777,rs479777,chr11,62606161,62606405,ENSG00000254964,1,ENSG00000254964
4,chr11,62605000,62610000,chr11,64105000,64110000,.,1.006396e-07,.,.,...,64107476,64107477,rs479777,rs479777,chr11,62602218,62612775,EML3,1,ENSG00000149499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,chr6,33545000,33550000,chr6,34855000,34860000,.,1.161047e-07,.,.,...,33548089,33548090,rs3216621,rs3216621:33548090:A:AG,chr6,34792015,34883138,UHRF1BP1,1,ENSG00000065060
91,chr6,33545000,33550000,chr6,35225000,35230000,.,9.077886e-08,.,.,...,33548089,33548090,rs3216621,rs3216621:33548090:A:AG,chr6,33540046,33589026,GGNBP1,1,ENSG00000204188
92,chr6,33545000,33550000,chr6,35225000,35230000,.,9.077886e-08,.,.,...,33548089,33548090,rs3216621,rs3216621:33548090:A:AG,chr6,35213956,35253079,SCUBE3,1,ENSG00000146197
93,chr6,33545000,33550000,chr6,35225000,35230000,.,9.077886e-08,.,.,...,33548089,33548090,rs3216621,rs3216621:33548090:A:AG,chr6,33540046,33589026,GGNBP1,1,ENSG00000204188


In [106]:
gs_loops.columns = bedpe_10cols + ['chrS', 'startS', 'endS', 'rs_id', 'variant_id'] + \
                                  ['chrG', 'startG', 'endG', 'gene_name', 'gene_num', 'gene_id']

In [107]:
gs_loops

,chrA,startA,endA,chrB,startB,endB,name,score,strand1,strand2,...,startS,endS,rs_id,variant_id,chrG,startG,endG,gene_name,gene_num,gene_id
0,chr1,192535000,192540000,chr1,193025000,193030000,.,3.378864e-07,.,.,...,192538495,192538496,rs2760530,rs2760530,chr1,192517190,192538102,ENSG00000236069,1,ENSG00000236069
1,chr1,192535000,192540000,chr1,193025000,193030000,.,3.378864e-07,.,.,...,192538495,192538496,rs2760530,rs2760530,chr1,192167786,192957713,ENSG00000285280,1,ENSG00000285280
2,chr1,192535000,192540000,chr1,193025000,193030000,.,3.378864e-07,.,.,...,192538495,192538496,rs2760530,rs2760530,chr1,193012250,193060080,UCHL5,1,ENSG00000116750
3,chr11,62605000,62610000,chr11,64105000,64110000,.,1.006396e-07,.,.,...,64107476,64107477,rs479777,rs479777,chr11,62606161,62606405,ENSG00000254964,1,ENSG00000254964
4,chr11,62605000,62610000,chr11,64105000,64110000,.,1.006396e-07,.,.,...,64107476,64107477,rs479777,rs479777,chr11,62602218,62612775,EML3,1,ENSG00000149499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,chr6,33545000,33550000,chr6,34855000,34860000,.,1.161047e-07,.,.,...,33548089,33548090,rs3216621,rs3216621:33548090:A:AG,chr6,34792015,34883138,UHRF1BP1,1,ENSG00000065060
91,chr6,33545000,33550000,chr6,35225000,35230000,.,9.077886e-08,.,.,...,33548089,33548090,rs3216621,rs3216621:33548090:A:AG,chr6,33540046,33589026,GGNBP1,1,ENSG00000204188
92,chr6,33545000,33550000,chr6,35225000,35230000,.,9.077886e-08,.,.,...,33548089,33548090,rs3216621,rs3216621:33548090:A:AG,chr6,35213956,35253079,SCUBE3,1,ENSG00000146197
93,chr6,33545000,33550000,chr6,35225000,35230000,.,9.077886e-08,.,.,...,33548089,33548090,rs3216621,rs3216621:33548090:A:AG,chr6,33540046,33589026,GGNBP1,1,ENSG00000204188


## Summary of loop intersections 

#### Number of loops per SNP

In [108]:
gs_loops.rs_id.value_counts()

rs9467740    34
rs1131017    25
rs3216621    24
rs479777      6
rs2289702     3
rs2760530     3
Name: rs_id, dtype: int64

#### Number of loops per gene

In [109]:
gs_loops.gene_name.value_counts()

BTN2A2             14
TIMELESS           11
GGNBP1             10
PPT2                2
H2BC13              2
OR2B7P              2
EGFL8               2
RPL35AP4            2
RPL13P              2
FLRT1               2
UHRF1BP1            2
H2BC16P             2
ESYT1               2
ENSG00000257553     2
ENSG00000284607     2
LINC01623           2
PA2G4               2
H2AC13              2
ANKRD34C-AS1        2
PPT2-EGFL8          2
SCUBE3              2
ABT1                2
ZSCAN16-AS1         2
H4C10P              2
MACROD1             2
CYCSP55             2
TAC3                1
HNRNPCP3            1
EML3                1
ENSG00000285280     1
ENSG00000212383     1
UCHL5               1
ENSG00000286069     1
ENSG00000257411     1
MYO1A               1
ENSG00000258679     1
BAZ2A               1
ENSG00000236069     1
ENSG00000254964     1
ENSG00000245651     1
Name: gene_name, dtype: int64

## Convert all results to output and WashU visualization results

In [226]:
def bedpe_to_WashU_longrange(fn, df):
    """
        Convert from a loop bedpe file into WashU longrange, 
        includes bgzip and tabix of the fn. 
        
        Params
        -------
        fn: str
            path to the longrange output file (without gz)
            
        df: dataframe
            columns 1-6 are as expected and column 8 is the p or q-value. 
            
        Output
        ------
        gzfn: str
            path to the longrange with bgzip compression
        tabix_fn: str
            path to the index of the longrange file
            
    """

    # parsing the data into WashU longrage format
    data = []
    for sr in df.values.tolist():

        # calculate the -log(FDR)
        qval = -np.log(sr[7])

        # get the first pair data
        second_pair_str = '{}:{}-{},{:.5f}'.format(*sr[3:6], qval)
        first_row = sr[0:3] + [second_pair_str]

        # get the second pair data
        first_pair_str = '{}:{}-{},{:.5f}'.format(*sr[0:3], qval)
        second_row = sr[3:6] + [first_pair_str]

        # add each data row
        data.append(first_row)
        data.append(second_row)

    data = sorted(data, key=lambda x: (x[0], x[1], x[2]))

    # writing out the data
    with open(fn, 'w') as f:
        for line in data:
            info = [str(x) for x in line]
            info = '\t'.join(info)
            f.write(info + '\n')
            
    # run bgzip
    cmd = '{} {}'.format(bgzip, lrange_fn)
    print(cmd)
    job = sp.Popen(cmd, stderr=sp.PIPE,stdout=sp.PIPE, shell=True)

    out, err = job.communicate()
    print('out:', out.decode())
    print('err:', err.decode())
    
    # run tabix
    lrange_gzfn = os.path.join(outdir, 'gs_loops.longrange.bed.gz')
    cmd = '{} {}'.format(tabix, lrange_gzfn)
    print(cmd)
    job = sp.Popen(cmd, stderr=sp.PIPE,stdout=sp.PIPE, shell=True)

    out, err = job.communicate()
    print('out:', out.decode())
    print('err:', err.decode())

    print('Created the gzfn: {}'.format(fn + '.gz'))
    print('Created the tabix: {}'.format(fn + '.gz.tbi'))

In [229]:
# make the longrange file
lrange_fn = os.path.join(outdir, 'gs_loops.longrange.bed')
bedpe_to_WashU_longrange(lrange_fn, gs_loops)

/mnt/BioApps/tabix/tabix-0.2.6/bgzip results/main/loop_analysis/washU/gs_loops.longrange.bed
out: 
err: 
/mnt/BioApps/tabix/tabix-0.2.6/tabix results/main/loop_analysis/washU/gs_loops.longrange.bed.gz
out: 
err: 
Created the gzfn: results/main/loop_analysis/washU/gs_loops.longrange.bed.gz
Created the tabix: results/main/loop_analysis/washU/gs_loops.longrange.bed.gz.tbi


In [249]:
gs_loops.gene_name.unique()

array(['ENSG00000236069', 'ENSG00000285280', 'UCHL5', 'ENSG00000254964',
       'EML3', 'MACROD1', 'FLRT1', 'ENSG00000286069', 'TIMELESS',
       'ENSG00000257411', 'PA2G4', 'ENSG00000257553', 'ESYT1', 'BAZ2A',
       'ENSG00000212383', 'ENSG00000258679', 'MYO1A', 'TAC3',
       'ENSG00000245651', 'ANKRD34C-AS1', 'HNRNPCP3', 'BTN2A2',
       'ENSG00000284607', 'ABT1', 'H4C10P', 'H2BC13', 'H2AC13', 'H2BC16P',
       'OR2B7P', 'ZSCAN16-AS1', 'RPL13P', 'LINC01623', 'EGFL8', 'PPT2',
       'PPT2-EGFL8', 'GGNBP1', 'RPL35AP4', 'CYCSP55', 'UHRF1BP1',
       'SCUBE3'], dtype=object)